**Переда запуском добавь коко2014 [датасет](https://www.kaggle.com/datasets/nadaibrahim/coco2014) в ноутбук)**

In [1]:
!pip install wandb
!pip install bitsandbytes
!pip install ruclip==0.0.2
!pip install transformers==4.27.4
!pip install pycocotools
!pip install git+https://github.com/openai/CLIP.git
!pip install open_clip_torch

  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/d9/8d/b62d4fb02587e293e5b91b68bbcaa2d88c6a0360b622e9521d4bd07a20cd/bitsandbytes-0.41.3.post2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=193020 sha256=7cab2c2bbe6fbd7f3cfca2cebe8fc1d58880efd2621a39b1b3f54519969e9922
  Stored in directory: /root/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome
  Attempting uninstall: more-itertools
    Found existing installation: more-itertools 10.1.0
    U

#### Качаем РУССКИЙ капчеринг

In [2]:
#!rm -rf ru_capt.json
##!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uIO34T8d0ML23I30mcRFAD7niggm1kIt' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uIO34T8d0ML23I30mcRFAD7niggm1kIt" -O ru_capt.json && rm -rf /tmp/cookies.txt

#### Качаем ембединги датасета

In [3]:
#!rm -rf train2014_emb.zip
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=11OpNsXhmtafzItkGxaTqMXqgL2dQycl6' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=11OpNsXhmtafzItkGxaTqMXqgL2dQycl6" -O train2014_emb.zip && rm -rf /tmp/cookies.txt
# !unzip train2014_emb.zip

In [4]:
#!rm -rf val2014_emb.zip
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1a-XuH0q5Ktlo6fIGFKQVkGEUkjiQ10X0' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1a-XuH0q5Ktlo6fIGFKQVkGEUkjiQ10X0" -O val2014_emb.zip && rm -rf /tmp/cookies.txt
# !unzip val2014_emb.zip


In [5]:
#import torchvision.datasets as dset
#coco_train = dset.CocoDetection(
#    root = "/kaggle/input/coco2014/train2014/train2014",
#    annFile = "/kaggle/input/coco2014/captions/annotations/captions_train2014.json"
#)
#coco_val = dset.CocoDetection(
#    root = "/kaggle/input/coco2014/val2014/val2014",
#    annFile = "/kaggle/input/coco2014/captions/annotations/captions_val2014.json"
#)

In [6]:
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader


In [7]:
from tqdm import tqdm, trange
import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
from torch.cuda.amp import autocast

import ruclip
import clip, open_clip
import random

In [8]:
from transformers import GPT2Config, GPT2Model
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [9]:
from transformers import AdamW, get_linear_schedule_with_warmup

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [10]:
#from multilingual_clip import pt_multilingual_clip

In [11]:
manualSeed = 1337
#manualSeed = random.randint(1, 10000) # use if you want new results
random.seed(manualSeed)
torch.manual_seed(manualSeed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
    
    @autocast()  
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [13]:
class MlpTransformer(nn.Module):
    def __init__(self, in_dim, h_dim, out_d: Optional[int] = None, act=nnf.relu, dropout=0.):
        super().__init__()
        out_d = out_d if out_d is not None else in_dim
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.act = act
        self.fc2 = nn.Linear(h_dim, out_d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [14]:
class MultiHeadAttention(nn.Module):

    def __init__(self, dim_self, dim_ref, num_heads, bias=True, dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim_self // num_heads
        self.scale = head_dim ** -0.5
        self.to_queries = nn.Linear(dim_self, dim_self, bias=bias)
        self.to_keys_values = nn.Linear(dim_ref, dim_self * 2, bias=bias)
        self.project = nn.Linear(dim_self, dim_self)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y=None, mask=None):
        y = y if y is not None else x
        b, n, c = x.shape
        _, m, d = y.shape
        # b n h dh
        queries = self.to_queries(x).reshape(b, n, self.num_heads, c // self.num_heads)
        # b m 2 h dh
        keys_values = self.to_keys_values(y).reshape(b, m, 2, self.num_heads, c // self.num_heads)
        keys, values = keys_values[:, :, 0], keys_values[:, :, 1]
        attention = torch.einsum('bnhd,bmhd->bnmh', queries, keys) * self.scale
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(1)
            attention = attention.masked_fill(mask.unsqueeze(3), float("-inf"))
        attention = attention.softmax(dim=2)
        out = torch.einsum('bnmh,bmhd->bnhd', attention, values).reshape(b, n, c)
        out = self.project(out)
        return out, attention

In [15]:
class TransformerLayer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        x_, attention = self.attn(self.norm1(x), y, mask)
        x = x + x_
        x = x + self.mlp(self.norm2(x))
        return x, attention

    def forward(self, x, y=None, mask=None):
        x = x + self.attn(self.norm1(x), y, mask)[0]
        x = x + self.mlp(self.norm2(x))
        return x

    def __init__(self, dim_self, dim_ref, num_heads, mlp_ratio=4., bias=False, dropout=0., act=nnf.relu,
                 norm_layer: nn.Module = nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim_self)
        self.attn = MultiHeadAttention(dim_self, dim_ref, num_heads, bias=bias, dropout=dropout)
        self.norm2 = norm_layer(dim_self)
        self.mlp = MlpTransformer(dim_self, int(dim_self * mlp_ratio), act=act, dropout=dropout)


class Transformer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        attentions = []
        for layer in self.layers:
            x, att = layer.forward_with_attention(x, y, mask)
            attentions.append(att)
        return x, attentions

    def forward(self, x, y=None, mask=None):
        for i, layer in enumerate(self.layers):
            if i % 2 == 0 and self.enc_dec: # cross
                x = layer(x, y)
            elif self.enc_dec:  # self
                x = layer(x, x, mask)
            else:  # self or cross
                x = layer(x, y, mask)
        return x

    def __init__(self, dim_self: int, num_heads: int, num_layers: int, dim_ref: Optional[int] = None,
                 mlp_ratio: float = 2., act=nnf.relu, norm_layer: nn.Module = nn.LayerNorm, enc_dec: bool = False):
        super(Transformer, self).__init__()
        dim_ref = dim_ref if dim_ref is not None else dim_self
        self.enc_dec = enc_dec
        if enc_dec:
            num_layers = num_layers * 2
        layers = []
        for i in range(num_layers):
            if i % 2 == 0 and enc_dec:  # cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            elif enc_dec:  # self
                layers.append(TransformerLayer(dim_self, dim_self, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            else:  # self or cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
        self.layers = nn.ModuleList(layers)

In [16]:
class TransformerMapper(nn.Module):

    def forward(self, x):
        x = self.linear(x).view(x.shape[0], self.clip_length, -1)
        prefix = self.prefix_const.unsqueeze(0).expand(x.shape[0], *self.prefix_const.shape)
        prefix = torch.cat((x, prefix), dim=1)
        out = self.transformer(prefix)[:, self.clip_length:]
        return out

    def __init__(self, dim_clip: int, dim_embedding: int, prefix_length: int, clip_length: int, num_layers: int = 8):
        super(TransformerMapper, self).__init__()
        self.clip_length = clip_length
        self.transformer = Transformer(dim_embedding, 8, num_layers)
        self.linear = nn.Linear(dim_clip, clip_length * dim_embedding)
        self.prefix_const = nn.Parameter(torch.randn(prefix_length, dim_embedding), requires_grad=True)

In [17]:
def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=False,
):
    
    for name, p in model.named_parameters():
    # freeze all parameters except the layernorm and positional embeddings
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

In [18]:
from enum import Enum
class MappingType(Enum):
    MLP = 'mlp'
    Transformer = 'transformer'

In [19]:
gpt_model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
class ClipCaptionModel(nn.Module):
    def __init__(
        self,
        prefix_length: int,
        clip_length: Optional[int] = None,
        prefix_size: int = 640,
        num_layers: int = 8,
        mapping_type: MappingType = MappingType.MLP
    ):
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length

        self.gpt = GPT2LMHeadModel.from_pretrained(gpt_model_name)
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]

        if mapping_type == MappingType.MLP:
            self.clip_project = MLP((
                prefix_size,
                self.gpt_embedding_size * prefix_length // 2,
                self.gpt_embedding_size * prefix_length
            ))
        else:
            self.clip_project = TransformerMapper(
                prefix_size,
                self.gpt_embedding_size,
                prefix_length,
                clip_length, 
                num_layers
            )

        
    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    @autocast() 
    def forward(
        self,        
        tokens: torch.Tensor,
        prefix: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(
            prefix.float()
        ).view(-1, self.prefix_length, self.gpt_embedding_size)

        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)

        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask, output_hidden_states = True)
        
        return out

In [20]:
class ClipCaptionPrefix(ClipCaptionModel):
    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

In [21]:
path_emb_train_coco = f"/kaggle/input/coco-ru-image-and-capt/Features_train_coco_ru_vitb16_82783.pkl"
path_emb_val_coco = f"/kaggle/input/coco-ru-image-and-capt/Features_val_coco_ru_vitb16.pkl"

In [22]:
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)


In [23]:
class ClipCocoDataset(Dataset):
    def __init__(
        self,
        data_path: str,
        prefix_length=30,
        model_type = gpt_model_name,
        normalize_prefix=False,
        train=True,
    ):

        self.tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        if train:
            with open(data_path, 'rb') as f:
                all_data = CPU_Unpickler(f).load() #pickle.load(f)
            print("Data size is %0d" % len(all_data["clip_embedding"]))
        else:
            with open(data_path, 'rb') as f:
                all_data = CPU_Unpickler(f).load() #pickle.load(f)
            print("Data size is %0d" % len(all_data["clip_embedding"]))

        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        
        self.captions = captions_raw

        self.image_id = all_data["image_id"]

        self.captions_tokens = []
        self.caption2embedding = []
        max_seq_len = 0
        i = 0
        for caption in tqdm(captions_raw):
            self.captions_tokens.append(
                torch.tensor(self.tokenizer.encode(caption), dtype=torch.int64)
            )
            self.caption2embedding.append(self.prefixes[i])
            i += 1
            max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])

        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))

    def get_image(self, item):
        if self.train:
            path_img = f"/kaggle/input/train2014/train2014/COCO_train2014_{self.image_id[item]}.jpg"
        else:
            path_img = f"/kaggle/input/val2014/val2014/COCO_val2014_{self.image_id[item]}.jpg"
            
        image = cv2.imread(path_img)
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        image.thumbnail((196, 196), Image.Resampling.LANCZOS)
        return image
    
    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            #self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            #self.captions_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask
    
    def __len__(self) -> int:
        return len(self.captions_tokens)

    def __getitem__(self, item):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[item]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

In [24]:
def save_config(args: argparse.Namespace):
    config = {}
    for key, item in args._get_kwargs():
        config[key] = item
    out_path = os.path.join(args.out_dir, f"{args.prefix}.json")
    with open(out_path, 'w') as outfile:
        json.dump(config, outfile)


def load_model(config_path: str, epoch_or_latest: Union[str, int] = '_latest'):
    with open(config_path) as f:
        config = json.load(f)
    parser = argparse.ArgumentParser()
    parser.set_defaults(**config)
    args = parser.parse_args()
    if type(epoch_or_latest) is int:
        epoch_or_latest = f"-{epoch_or_latest:03d}"
    model_path = os.path.join(args.out_dir, f"{args.prefix}{epoch_or_latest}.pt")
    if args.only_prefix:
        model = ClipCaptionPrefix(args.prefix_length)
    else:
        model = ClipCaptionModel(args.prefix_length)
    if os.path.isfile(model_path):
        print(f"loading model from {model_path}")
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    else:
        print(f"{model_path} is not exist")
    return model, parser


In [25]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [26]:
torch.cuda.mem_get_info()[1] / 1024**3

15.8992919921875

In [27]:
import bitsandbytes as bnb

## **TRAIN**

**Запусти ячейку и появится окошко для ввода токена)**

In [29]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [30]:
import wandb

wandb.init(project="ClipCap_NAS", name="Selfsupervised&GPT-2-BLEU")

wandb: Currently logged in as: sanchoues (clipcap_nas). Use `wandb login --relogin` to force relogin


In [31]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers.optimization import Adafactor, AdafactorSchedule

import os
import pickle
import sys
import argparse

from typing import Tuple, Optional, Union
from torch.cuda.amp import autocast


In [32]:
import nltk
import numpy as np

In [33]:
from datasets import load_dataset, load_metric

In [34]:
from nltk.translate.bleu_score import corpus_bleu

def calc_bleu(y_pred, y_true):
    references = [[reference.split()] for reference in y_true]
    hypotheses = [hypothesis.split() for hypothesis in y_pred]
    # Рассчитываем BLEU-4
    bleu_score = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
    return bleu_score*100

In [35]:
def sup_train(
    train_dataset: ClipCocoDataset,
    train_dataloader,
    #valid_dataset: ClipCocoDataset,
    model: ClipCaptionModel,
    optimizer,
    scheduler,
    args,
    warmup_steps: int = 5000,
    output_dir: str = ".",
    output_prefix: str = ""
):
    #
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = freeze(model)


    model.train()

    mean_epoch_train_loss = []
    mean_bleu_train_epoch = []
    
    
    for epoch in range(epochs):
        loss_train_epoch = []
        bleu_train_epoch = []
        print(f">>> Training epoch {epoch+1}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        step=0
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            step += 1
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.bfloat16)
            
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, train_dataset.prefix_length - 1: -1]

            loss = nnf.cross_entropy(
                logits.reshape(-1, logits.shape[-1]),
                tokens.flatten().to(torch.int64),
                ignore_index=0
            )

            loss.backward()
            optimizer.step()
            scheduler.step()

            optimizer.zero_grad()
            progress.set_postfix({"sup_loss_train": loss.item()})
            

            loss_train_epoch.append(loss.item())

            if step % 500 == 0:
                wandb.log({"sup_loss_train":  loss.item()})
            
            if step % 1000 == 0:
                with torch.no_grad():
                    # BLEU-4
                    logits_cpu = logits.cpu()
                    tokens_cpu = tokens.cpu()
                    generated_texts = []
                    real_text = []
                    for b in range(batch_size):
                        generated_text_batch = train_dataset.tokenizer.decode(logits_cpu[b].argmax(dim=-1).tolist())
                        first_dot_index = generated_text_batch.find('.')
                        if first_dot_index != -1:
                            generated_texts.append(generated_text_batch[35:first_dot_index + 1])
                        else:
                            generated_texts.append(generated_text_batch[35:])
                        
                        real_text_batch = train_dataset.tokenizer.decode(tokens_cpu[b].tolist())
                        first_pad_index = real_text_batch.find('<pad>')
                        if first_pad_index != -1:
                            real_text.append(real_text_batch[35:first_pad_index])
                        else:
                            real_text.append(real_text_batch[35:])
                    
                    bleu = calc_bleu(generated_texts, real_text)
                    wandb.log({"bleu-4 train":  bleu})
                    bleu_train_epoch.append(bleu)
                    

            progress.update()
            if (idx + 1) % 7000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest_gpt2_medium.pt"),
                )
        progress.close()
        if epoch % args.save_every == 0:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{(epoch+1):03d}_gpt2_medium.pt"),
            )
        mean_epoch_train_loss.append(np.mean(loss_train_epoch))
        mean_bleu_train_epoch.append(np.mean(bleu_train_epoch))
        
        wandb.log({"mean_epoch_sup_train_loss": mean_epoch_train_loss[-1]})
        wandb.log({"mean_bleu_sup_train_epoch": mean_bleu_train_epoch[-1]})

    return model



In [36]:
class Args():
    def __init__(self):
        self.backbone = gpt_model_name
        self.train_data = "/kaggle/input/coco-ru-image-and-capt/Features_train_coco_ru_vitb16_82783.pkl"
        self.valid_data = "/kaggle/input/coco-ru-image-and-capt/Features_val_coco_ru_vitb16.pkl"
        self.out_dir = 'checkpoints'
        self.prefix = '20bs_adamw'
        self.epochs = 1
        self.save_every = 1
        self.prefix_length = 30
        self.bs = 3
        self.only_prefix = False
        self.lr = 2e-5
        self.warmup_steps = 5000
args = Args()

In [37]:
train_dataset = ClipCocoDataset(args.train_data, args.prefix_length, train=True)
#valid_dataset = ClipCocoDataset(args.valid_data, args.prefix_length, train=False)

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Data size is 414113


100%|██████████| 414113/414113 [02:40<00:00, 2582.38it/s]


In [38]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BGVig1avA29Ul8M1kvQPn66wv0q6CZFz' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BGVig1avA29Ul8M1kvQPn66wv0q6CZFz" -O last_weights.zip && rm -rf /tmp/cookies.txt
!unzip last_weights.zip

--2023-12-12 17:21:23--  https://docs.google.com/uc?export=download&confirm=t&id=1BGVig1avA29Ul8M1kvQPn66wv0q6CZFz
Resolving docs.google.com (docs.google.com)... 74.125.134.101, 74.125.134.139, 74.125.134.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.134.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/trnh9msei7o4c7qf9v5eq1q7qqgk95sp/1702401675000/17733576292032671640/*/1BGVig1avA29Ul8M1kvQPn66wv0q6CZFz?e=download&uuid=4a40971e-8d1e-4778-90db-7722f3efb246 [following]
--2023-12-12 17:21:23--  https://doc-10-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/trnh9msei7o4c7qf9v5eq1q7qqgk95sp/1702401675000/17733576292032671640/*/1BGVig1avA29Ul8M1kvQPn66wv0q6CZFz?e=download&uuid=4a40971e-8d1e-4778-90db-7722f3efb246
Resolving doc-10-3g-docs.googleusercontent.com (doc-10-3g-docs.googleusercontent.com)... 172.217.193.132,

In [39]:
wandb.config = {
  "learning_rate": args.lr,
  "epochs": args.epochs,
  "batch_size": args.bs
}



model = ClipCaptionModel(args.prefix_length)
model_path = '20bs_adamw-001_gpt2_medium.pt'
model.load_state_dict(torch.load(model_path, map_location='cpu')) 
model = model.to(device)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [40]:
len(train_dataset)

414113

In [41]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

def sup_unsup_train_dataset(dataset, val_split):
    sup_idx, unsup_idx = train_test_split(list(range(len(dataset))), test_size=1-val_split, shuffle=False)
    return Subset(dataset, sup_idx), Subset(dataset, unsup_idx)

sup_train_dataset, unsup_train_dataset = sup_unsup_train_dataset(train_dataset, 0.3)

sup_train_dataset.prefix_length = train_dataset.prefix_length
unsup_train_dataset.prefix_length = train_dataset.prefix_length
sup_train_dataset.tokenizer = train_dataset.tokenizer
unsup_train_dataset.tokenizer = train_dataset.tokenizer

print(len(sup_train_dataset))
print(len(unsup_train_dataset))

124233
289880


In [42]:
sup_train_dataloader = DataLoader(
    sup_train_dataset,
    batch_size=args.bs,
    shuffle=True,
    drop_last=True,
)

In [46]:
# sup_optimizer = AdamW(
#         model.parameters(),
#         lr=args.lr,
#     )

# sup_scheduler = get_linear_schedule_with_warmup(
#     sup_optimizer,
#     num_warmup_steps=args.warmup_steps,
#     num_training_steps=args.epochs * len(sup_train_dataloader)
# )


In [46]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

20bs_adamw:   4%|▍         | 4205/96626 [22:10<8:07:28,  3.16it/s, unsup_loss_train=-87.6]


In [42]:
sys.stdout.flush()

In [48]:
# print("Train both prefix and GPT2")
# sys.stdout.flush()
# model = sup_train(
#     sup_train_dataset,
#     sup_train_dataloader,
#     model,
#     sup_optimizer,
#     sup_scheduler,
#     args,
#     warmup_steps=args.warmup_steps,
#     output_dir=args.out_dir,
#     output_prefix=args.prefix
# )

Train both prefix and GPT2
>>> Training epoch 1


20bs_adamw:   2%|▏         | 999/41411 [03:48<2:32:08,  4.43it/s, sup_loss_train=2.15]/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
20bs_adamw:   5%|▍         | 1999/41411 [07:34<2:28:36,  4.42it/s, sup_loss_train=1.72] /opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
20bs_adamw:  10%|▉         | 3999/41411 [15:06<2:20:44,  4.43it/s, sup_loss_train=2.24] /opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
20bs_adamw: 100%|██████████| 41411/41411 [2:37:10<00:00,  4.39it/s, s

In [57]:
# with torch.no_grad():
#     for idx, (tokens, mask, prefix) in enumerate(sup_train_dataloader):
#         model.zero_grad()
#         tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.bfloat16)

#         outputs = model(tokens, prefix, mask)
#         logits = outputs.logits[:, train_dataset.prefix_length - 1: -1]

#         # BLEU-4
#         logits_cpu = logits.cpu()
#         tokens_cpu = tokens.cpu()
#         generated_texts = []
#         real_text = []
#         for b in range(args.bs):
#             generated_text_batch = train_dataset.tokenizer.decode(logits_cpu[b].argmax(dim=-1).tolist())
#             first_dot_index = generated_text_batch.find('.')
#             if first_dot_index != -1:
#                 generated_texts.append(generated_text_batch[35:first_dot_index + 1])
#             else:
#                 generated_texts.append(generated_text_batch[35:])

#             real_text_batch = train_dataset.tokenizer.decode(tokens_cpu[b].tolist())
#             first_pad_index = real_text_batch.find('<pad>')
#             if first_pad_index != -1:
#                 real_text.append(real_text_batch[35:first_pad_index])
#             else:
#                 real_text.append(real_text_batch[35:])
                
#             print('Real: ', real_text_batch)
#             print('Generated: ', generated_texts)
#             print('Bleu:', calc_bleu(generated_texts, real_text))
#             break
#         break

Real:  Вопрос: что на изображении? Ответ: Два высоких жирафа гуляют по своей зоовыставке.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Generated:  ['Два жира жирафа,ляют по полю территорииозащитставке.']
Bleu: 64.31870218238024


In [50]:
#!ls /kaggle/working/checkpoints

20bs_adamw-001_gpt2_medium.pt  20bs_adamw_latest_gpt2_medium.pt


In [ ]:
#!rm /kaggle/working/first_start-003_gpt2_medium.pt

In [51]:
#!mv /kaggle/working/checkpoints/20bs_adamw-001_gpt2_medium.pt /kaggle/working/


In [52]:
#!zip weights.zip 20bs_adamw-001_gpt2_medium.pt

  adding: 20bs_adamw-001_gpt2_medium.pt (deflated 22%)


In [43]:
question_tokens = torch.tensor(train_dataset.tokenizer.encode("Вопрос: что на изображении? Ответ:"), dtype=torch.int64)
question_tokens = torch.cat((question_tokens, torch.zeros(train_dataset.max_seq_len - question_tokens.shape[0], dtype=torch.int64) - 1))
q_mask = question_tokens.ge(0)
question_tokens[~q_mask] = 0
q_mask = q_mask.float()
q_mask = torch.cat((torch.ones(train_dataset.prefix_length), q_mask), dim=0)
question_tokens = torch.stack([question_tokens] * args.bs).to(device)
q_mask = torch.stack([q_mask] * args.bs).to(device)


In [44]:
unsup_train_dataloader = DataLoader(
    unsup_train_dataset,
    batch_size=args.bs,
    shuffle=True,
    drop_last=True,
)

In [45]:
unsup_optimizer = AdamW(
        model.parameters(),
        lr=args.lr,
    )

unsup_scheduler = get_linear_schedule_with_warmup(
    unsup_optimizer,
    num_warmup_steps=args.warmup_steps,
    num_training_steps=args.epochs * len(unsup_train_dataloader)
)


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [46]:
import torch.nn.functional as F


def unsup_train(
    train_dataset: ClipCocoDataset,
    train_dataloader,
    #valid_dataset: ClipCocoDataset,
    model: ClipCaptionModel,
    optimizer,
    scheduler,
    args,
    warmup_steps: int = 5000,
    output_dir: str = ".",
    output_prefix: str = ""
):
    #
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = freeze(model)


    model.train()

    mean_epoch_train_loss = []
    mean_bleu_train_epoch = []
    
    
    for epoch in range(epochs):
        loss_train_epoch = []
        bleu_train_epoch = []
        print(f">>> Training epoch {epoch+1}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        step=0
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            step += 1
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.bfloat16)
            
            outputs = model(question_tokens, prefix, q_mask)
            
            last_hiddens_state = outputs.hidden_states[-1][:, :model.prefix_length]
            image_emb = model.clip_project(prefix.float()).view(-1, model.prefix_length, model.gpt_embedding_size)
            loss = - torch.mean(F.cosine_similarity(image_emb,
                                                   last_hiddens_state, dim=-1))

            loss.backward()
            optimizer.step()
            scheduler.step()

            optimizer.zero_grad()
            progress.set_postfix({"unsup_loss_train": loss.item()})
            

            loss_train_epoch.append(loss.item())

            if step % 500 == 0:
                wandb.log({"unsup_loss_train":  loss.item()})
            
            if step % 1000 == 0:
                with torch.no_grad():
                    # BLEU-4
                    logits = outputs.logits[:, train_dataset.prefix_length - 1: -1]
                    logits_cpu = logits.cpu()
                    tokens_cpu = tokens.cpu()
                    generated_texts = []
                    real_text = []
                    for b in range(batch_size):
                        generated_text_batch = train_dataset.tokenizer.decode(logits_cpu[b].argmax(dim=-1).tolist())
                        first_dot_index = generated_text_batch.find('.')
                        if first_dot_index != -1:
                            generated_texts.append(generated_text_batch[35:first_dot_index + 1])
                        else:
                            generated_texts.append(generated_text_batch[35:])
                        
                        real_text_batch = train_dataset.tokenizer.decode(tokens_cpu[b].tolist())
                        first_pad_index = real_text_batch.find('<pad>')
                        if first_pad_index != -1:
                            real_text.append(real_text_batch[35:first_pad_index])
                        else:
                            real_text.append(real_text_batch[35:])
                    
                    bleu = calc_bleu(generated_texts, real_text)
                    wandb.log({"bleu-4 unsupervised train":  bleu})
                    bleu_train_epoch.append(bleu)
                    

            progress.update()
            if (idx + 1) % 7000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest_gpt2_medium.pt"),
                )
        progress.close()
        if epoch % args.save_every == 0:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{(epoch+1):03d}_gpt2_medium.pt"),
            )
        mean_epoch_train_loss.append(np.mean(loss_train_epoch))
        mean_bleu_train_epoch.append(np.mean(bleu_train_epoch))
        
        wandb.log({"mean_epoch_unsup_train_loss": mean_epoch_train_loss[-1]})
        wandb.log({"mean_bleu_unsup_train_epoch": mean_bleu_train_epoch[-1]})

    return model


In [47]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print("Train both prefix and GPT2")
sys.stdout.flush()
model = unsup_train(
    unsup_train_dataset,
    unsup_train_dataloader,
    model,
    unsup_optimizer,
    unsup_scheduler,
    args,
    warmup_steps=args.warmup_steps,
    output_dir=args.out_dir,
    output_prefix=args.prefix
)

Train both prefix and GPT2
>>> Training epoch 1


20bs_adamw:   4%|▍         | 3769/96626 [17:50<7:24:19,  3.48it/s, unsup_loss_train=-.963]

In [ ]:
with torch.no_grad():
    for idx, (tokens, mask, prefix) in enumerate(unsup_train_dataloader):
        model.zero_grad()
        tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.bfloat16)

        outputs = model(question_tokens, prefix, q_mask)
        logits = outputs.logits[:, train_dataset.prefix_length - 1: -1]
        
        # BLEU-4
        logits_cpu = logits.cpu()
        tokens_cpu = tokens.cpu()
        generated_texts = []
        real_text = []
        for b in range(args.bs):
            generated_text_batch = train_dataset.tokenizer.decode(logits_cpu[b].argmax(dim=-1).tolist())
            first_dot_index = generated_text_batch.find('.')
            if first_dot_index != -1:
                generated_texts.append(generated_text_batch[35:first_dot_index + 1])
            else:
                generated_texts.append(generated_text_batch[35:])

            real_text_batch = train_dataset.tokenizer.decode(tokens_cpu[b].tolist())
            first_pad_index = real_text_batch.find('<pad>')
            if first_pad_index != -1:
                real_text.append(real_text_batch[35:first_pad_index])
            else:
                real_text.append(real_text_batch[35:])
                
            print('Real: ', real_text_batch)
            print('Generated: ', generated_texts)
            print('Bleu:', calc_bleu(generated_texts, real_text))
            break
        break

In [ ]:
torch.save(
    model.state_dict(),
    os.path.join('checkpoints', "last_weights.pt"),
)

In [ ]:
!ls /kaggle/working/checkpoints

In [ ]:
!rm /kaggle/working/last_weights.pt
!rm /kaggle/working/weights.zip

In [ ]:
!mv /kaggle/working/checkpoints/last_weights.pt /kaggle/working/


In [ ]:
!zip weights.zip last_weights.pt